# 原地重新绘制Bokeh图表

In [2]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

调用`IPython.display.display_javascript()`运行JavaScript代码之后，该段代码保留在单元的输出中，可以调用`clear_output()`清除单元的输出。这样就可以在控件的回调函数中反复运行JavaScript代码，而不会造成Notebook逐渐变得庞大。

In [3]:
def call_javascript(code, delay=None):
    from IPython.display import display_javascript, clear_output    
    if delay is not None:
        code = 'window.setTimeout(function(){{{code}}}, {delay:d});'.format(code=code, delay=delay)
    display_javascript(code, raw=True)
    clear_output()

In [4]:
import re
from bokeh.io import show
from bokeh.charts import Line, Bar, Area, BoxPlot
from bokeh.embed import components
import pandas as pd
import numpy as np
import ipywidgets as iw
from IPython.display import display

In [5]:
df = pd.DataFrame(np.random.randn(100, 3), columns=["A", "B", "C"])

In [11]:
fig_settings = dict(
    id="myfig",
    plot_height=300
)

def plot_Line():
    return Line(df.cumsum(axis=0), x="index", **fig_settings)

def plot_BoxPlot():
    return BoxPlot(pd.melt(df), label="variable", values="value", **fig_settings)

def plot_Bar():
    return Bar(pd.melt(df), label="variable", values="value", legend=None, **fig_settings)
    
def on_click(button):
    name = button.description
    func = globals()["plot_{}".format(name)]
    fig = func()
    fig._id = "myfig"
    script, html = components(fig)
    script = re.search(r'<script type="text/javascript">(.+?)</script>', script, flags=re.DOTALL | re.MULTILINE).group(1)
    w_html.value = html
    call_javascript(script)
    
buttons = [iw.Button(description=label) for label in ["Line", "BoxPlot", "Bar"]]
for button in buttons:
    button.on_click(on_click)
w_html = iw.HTML(layout=iw.Layout(height="300px"))
display(iw.VBox([iw.HBox(buttons), w_html]))
on_click(buttons[0])